# 内置Fixture
cd ch4

在处理临时文件时，pytest内置的fixture可以识别命令行参数、在多个测试会话间通信、校验输出流、更改环境变量、审查错误警报等等。

## tmpdir和tmpdir_factory
负责在测试运行前创建临时文件目录，并在测试结束后删除。

单个测试用tmpdir，多个测试用tmpdir_factory。

tmpdir作用域是函数级别，tmpdir_factory的作用范围是会话级别。

pytest -q -s test_tmpdir.py::test_tmpdir_factory

### 在其他作用范围内使用临时目录
如果需要模块或类级别作用范围的目录，该怎么办？

可以利用tmpdir_factory创建一个fixture。

cd ch4/authors 查看conftest.py

pytest -sv authors/test_authors.py

## 使用pytestconfig
cd /ch4/pytestconfig/conftest.py

hook函数pytest_addoption添加配置选项

pytestconfig获取配置选项

pytest -s -q test_config.py::test_option

pytest -s -q --myopt test_config.py::test_option

pytest -s -q --myopt --foo baz test_config.py::test_option

pytest -sv test_config.py::test_fixtures_for_options

pytest -sv test_config.py::test_pytestconfig


## 使用cache
###  pytest 运行完用例之后会生成一个 .pytest_cache 的缓存文件夹，用于记录用例的ids和上一次失败的用例。

1. 跑自动化时经常会出现这样一个情况，一轮自动化跑完后零星出现了几个失败测试用例，无法断定失败的原因，所以可能需要重新跑一下失败的测试用例去调试，那我们要做的是就去修改脚本把那几个失败的测试用例筛选出来然后重新跑一遍；

2. 但是在pytest测试框架因为有缓存功能，就不需要手动去查找上次失败的测试用例并筛选执行。

3. pytest测试框架中执行完所有的测试用例后会在当前目录下生成 .pytest_cache ，里面就保存了上一次用例执行的信息。

### pytest测试框架中跟缓存有关的命令行参数

1. --last-failed ： 如重新运行上次运行结果为失败的测试用例， 如果没有执行失败的用例则执行所有的测试用例。【 --lf 】

场景：因为某次改动导致了多个测试用例失败了，那么我可以先执行一次全量测试找出所有失败的用例，待问题修复后，再次执行时带上这个选项就可以快速的进行验证：只执行失败用例验证失败用例是否已修复。

这个的实现原理是：pytest会在每次执行时将该次执行失败的所有用例存到测试文件同级目录下的名为.pytest_cache/v/cache/lastfailed的隐藏文件里，这样就清楚的标明了上次执行失败的用例。

2. --failed-first ：运行所有测试，但首先运行上次运行失败的测试（这可能会重新测试，从而导致重复的 fixture setup/teardown ）【 --ff 】

3. --new-first ：根据文件插件的时间，新的测试用例会先运行。【 --nf 】

3. --cache-show ：显示 .pytest_cache 文件内容，不会收集测试用例也不会执行测试用例

4. --cache-clear ：开始执行测试之前先清空 .pytest_cache 文件

### 文件夹详解
.pytest_cache/v/cache/lastfailed ：上一次运行失败的测试用例。

.pytest_cache/v/cache/nodeids ：上一次运行的所有测试用例（无论测试用例的执行结果通过还是失败）。

.pytest_cache/v/cache/stepwise ：测试用例的路径

.pytest_cache/v/cache/.gitignore ：pytest测试框架 .pytest_cache的自带的 .gitignore 文件。

.pytest_cache/v/cache/CACHEDIR.TAG ：pytest创建的缓存目录标签。

### 实战
ch4/cache/test_pass_fail.py

1. pytest -v --tb=no test_pass_fail.py
2. pytest -v --tb=no --failed-first test_pass_fail.py/pytest -v --tb=no --ff test_pass_fail.py
3. pytest -v --tb=no --last-failed test_pass_fail.py/pytest -v --tb=no --lf test_pass_fail.py

ch4/cache/test_few_failures.py

1. pytest -q test_few_failures.py
2. pytest -q "test_few_failures.py::test_a[1e+25-1e+23-1.1e+25]" 很麻烦如果想跑错误的用例
3. pytest -q --lf -l test_few_failures.py/pytest -q --lf --showlocals test_few_failures.py 使用-l，失败的原因更明显了

### pytest --cache-show 显示缓存目录

### cache的其他功能
ch4/cache/test_slower.py

执行：
pytest -q --cache-clear test_slower.py
pytest -q --tb=line test_slower.py
pytest -q --cache-show

改造后
ch4/cache/test_slower_2.py
pytest -q --cache-clear test_slower_2.py
pytest -q --tb=line test_slower_2.py
pytest -q --cache-show
